In [1]:
import numpy as np
import os
import re
import tensorflow as tf
from config import *
import pandas as pd

from tensorflow.python.framework import dtypes
from tensorflow.python.framework import random_seed

/Users/cb.hsu/anaconda/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import losses
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
offset = 30000
length = 30000
historyLength = 256

In [4]:
def readData(dataPath):
    rawData = pd.read_csv(dataPath)
    return rawData

In [5]:
def extractDataSet(rawData, offset, length, historyLength):
    dataSet = rawData.iloc[offset: offset+length+historyLength, 2:7].values
    return dataSet

In [6]:
def extractIO(dataSet,length,historyLength,stopWin,stopLost):
    X = np.zeros([length,historyLength,5])
    y = np.zeros([length])
    close = dataSet[:, 0]
    for i in range(historyLength,  length+historyLength):
        ##
        batch = dataSet[i-historyLength:i, 0:5]
        X[i-historyLength] = batch
        
        ##
        entryPrice = close[i]
        label = 0
        for futureK in range(i+1, length+historyLength):
            if(close[futureK] - entryPrice > stopWin):
                label = 1
                break
            elif(close[futureK] - entryPrice < - stopLost):
                label = 0
                break
        y[i-historyLength] = label
    return X,y

In [7]:
def normalize(X):
    normX = np.zeros(X.shape)
    for i in range(X.shape[0]):
        maxValue = X[i,:,1].max()
        minValue = X[i,:,2].min()
        maxVolume = X[i,:,4].max()
        minVolume = X[i,:,4].min()
        for j in range(4):
            normX[i,:,j] = (X[i,:,j] - minValue) / (maxValue - minValue)
        normX[i,:,4] = (X[i,:,4] - minVolume) / (maxVolume - minVolume)
    normX = np.nan_to_num(normX)
    return normX

In [8]:
def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

In [9]:
def shuffle(X,Y):
    np.random.seed(10)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

In [10]:
def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_shape=(shape[1], shape[2])))
    # output shape: (1, 1)
    model.add(Dense(1))
    ## model.compile(loss="mean_square_error", optimizer="adam")
    model.compile(loss=losses.mean_squared_error, optimizer="adam")
    model.summary()
    return model

In [11]:
rawData = readData('TXF1-Minute-Trade.txt')
print(rawData)

              Date      Time     Open     High      Low    Close  TotalVolume
0        1998/7/22  09:01:00   7950.0   7950.0   7950.0   7950.0            9
1        1998/7/22  09:03:00   7950.0   7950.0   7945.0   7950.0            5
2        1998/7/22  09:05:00   7945.0   7950.0   7945.0   7950.0           13
3        1998/7/22  09:06:00   7950.0   7950.0   7950.0   7950.0            5
4        1998/7/22  09:07:00   7935.0   7950.0   7935.0   7950.0            7
5        1998/7/22  09:08:00   7930.0   7930.0   7930.0   7930.0            2
6        1998/7/22  09:09:00   7910.0   7910.0   7910.0   7910.0            1
7        1998/7/22  09:10:00   7940.0   7940.0   7911.0   7911.0            2
8        1998/7/22  09:12:00   7915.0   7915.0   7915.0   7915.0            1
9        1998/7/22  09:13:00   7915.0   7915.0   7912.0   7912.0            2
10       1998/7/22  09:14:00   7911.0   7911.0   7911.0   7911.0            4
11       1998/7/22  09:15:00   7910.0   7911.0   7910.0   7911.0

In [12]:
dataSet = extractDataSet(rawData, offset, length, historyLength)
print(dataSet)

[[6.515e+03 6.515e+03 6.515e+03 6.515e+03 1.300e+01]
 [6.517e+03 6.517e+03 6.513e+03 6.513e+03 6.000e+00]
 [6.513e+03 6.513e+03 6.513e+03 6.513e+03 3.000e+00]
 ...
 [7.852e+03 7.852e+03 7.850e+03 7.850e+03 8.000e+00]
 [7.843e+03 7.850e+03 7.843e+03 7.850e+03 7.000e+00]
 [7.850e+03 7.850e+03 7.844e+03 7.844e+03 2.000e+00]]


In [13]:
X, y = extractIO(dataSet,length,historyLength,stopWin,stopLost)
print(y)
print(X)

[1. 1. 0. ... 0. 0. 0.]
[[[6.515e+03 6.515e+03 6.515e+03 6.515e+03 1.300e+01]
  [6.517e+03 6.517e+03 6.513e+03 6.513e+03 6.000e+00]
  [6.513e+03 6.513e+03 6.513e+03 6.513e+03 3.000e+00]
  ...
  [6.485e+03 6.485e+03 6.485e+03 6.485e+03 4.000e+00]
  [6.482e+03 6.485e+03 6.482e+03 6.485e+03 1.500e+01]
  [6.489e+03 6.489e+03 6.489e+03 6.489e+03 5.500e+01]]

 [[6.517e+03 6.517e+03 6.513e+03 6.513e+03 6.000e+00]
  [6.513e+03 6.513e+03 6.513e+03 6.513e+03 3.000e+00]
  [6.507e+03 6.507e+03 6.504e+03 6.504e+03 1.100e+01]
  ...
  [6.482e+03 6.485e+03 6.482e+03 6.485e+03 1.500e+01]
  [6.489e+03 6.489e+03 6.489e+03 6.489e+03 5.500e+01]
  [6.466e+03 6.470e+03 6.466e+03 6.470e+03 1.400e+01]]

 [[6.513e+03 6.513e+03 6.513e+03 6.513e+03 3.000e+00]
  [6.507e+03 6.507e+03 6.504e+03 6.504e+03 1.100e+01]
  [6.504e+03 6.504e+03 6.503e+03 6.503e+03 4.000e+00]
  ...
  [6.489e+03 6.489e+03 6.489e+03 6.489e+03 5.500e+01]
  [6.466e+03 6.470e+03 6.466e+03 6.470e+03 1.400e+01]
  [6.465e+03 6.475e+03 6.465e+03 6.4

In [14]:
normX = normalize(X)
print(normX)

[[[0.97674419 0.97674419 0.97674419 0.97674419 0.11009174]
  [1.         1.         0.95348837 0.95348837 0.04587156]
  [0.95348837 0.95348837 0.95348837 0.95348837 0.01834862]
  ...
  [0.62790698 0.62790698 0.62790698 0.62790698 0.02752294]
  [0.59302326 0.62790698 0.59302326 0.62790698 0.12844037]
  [0.6744186  0.6744186  0.6744186  0.6744186  0.49541284]]

 [[1.         1.         0.95348837 0.95348837 0.04587156]
  [0.95348837 0.95348837 0.95348837 0.95348837 0.01834862]
  [0.88372093 0.88372093 0.84883721 0.84883721 0.09174312]
  ...
  [0.59302326 0.62790698 0.59302326 0.62790698 0.12844037]
  [0.6744186  0.6744186  0.6744186  0.6744186  0.49541284]
  [0.40697674 0.45348837 0.40697674 0.45348837 0.11926606]]

 [[1.         1.         1.         1.         0.01834862]
  [0.92682927 0.92682927 0.8902439  0.8902439  0.09174312]
  [0.8902439  0.8902439  0.87804878 0.87804878 0.02752294]
  ...
  [0.70731707 0.70731707 0.70731707 0.70731707 0.49541284]
  [0.42682927 0.47560976 0.4268292

In [15]:
X_train, y_train, X_val, y_val = splitData(X, y, 0.3)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(21000, 256, 5)
(21000,)
(9000, 256, 5)
(9000,)


In [16]:
X_train_, y_train_ = shuffle(X_train, y_train)

In [17]:
model = buildManyToOneModel(X_train_.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10)                640       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 651
Trainable params: 651
Non-trainable params: 0
_________________________________________________________________


In [156]:
model.fit(X_train_, y_train_, epochs=1000, batch_size=128, validation_data=(X_val, y_val), callbacks=[callback])

Train on 21000 samples, validate on 9000 samples
Epoch 1/1000
21000/21000 [==============================] - 63s 3ms/step - loss: 0.1686 - val_loss: 0.1470
Epoch 2/1000
21000/21000 [==============================] - 59s 3ms/step - loss: 0.1388 - val_loss: 0.1413
Epoch 3/1000
21000/21000 [==============================] - 65s 3ms/step - loss: 0.1375 - val_loss: 0.1412
Epoch 4/1000
21000/21000 [==============================] - 65s 3ms/step - loss: 0.1375 - val_loss: 0.1412
Epoch 5/1000
  640/21000 [..............................] - ETA: 1:22 - loss: 0.1329

KeyboardInterrupt: 